# Map-Matching

https://developer.here.com/documentation/route-match/topics/what-is.html

https://developer.here.com/api-explorer/rest/locations_data_to_route/match-gpx-data-to-a-route

In [1]:
import requests
from geojson import LineString, Feature
from ipyleaflet import Map, GeoJSON, basemap_to_tiles

from credentials import APP_ID, APP_CODE

In [2]:
# GPX example from https://developer.here.com/api-explorer/rest/locations_data_to_route/match-gpx-data-to-a-route
body = '''\
<?xml version="1.0"?>
<gpx version="1.0"
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xmlns="http://www.topografix.com/GPX/1/0"
xsi:schemaLocation="http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd">
  <trk>
    <trkseg>
      <trkpt lat="51.10177" lon="0.39349"/>
      <trkpt lat="51.10181" lon="0.39335"/>
      <trkpt lat="51.10255" lon="0.39369"/>
      <trkpt lat="51.10398" lon="0.39466"/>
      <trkpt lat="51.10501" lon="0.39533"/>
    </trkseg>
  </trk>
</gpx>
'''

In [3]:
# real world example
body = open('data/bus_ride.gpx').read()

In [4]:
url = f'http://rme.cit.api.here.com/2/matchroute.json'
params = dict(
    app_id=APP_ID, 
    app_code=APP_CODE,
    filetype='GPX',
    routemode='car',
)
resp = requests.post(url, params=params, data=body)
j = resp.json()

In [5]:
j

{'RouteLinks': [{'linkId': -83749612,
   'functionalClass': 5,
   'confidence': 0.89,
   'shape': '54.3877 10.376 54.38728 10.3748 54.3871 10.37395 54.38716 10.37366',
   'offset': 0.48092,
   'mSecToReachLinkFromStart': 13447,
   'linkLength': 169.64000000000001},
  {'linkId': 759253714,
   'functionalClass': 4,
   'confidence': 0.96,
   'shape': '54.38716 10.37366 54.38733 10.3737 54.38744 10.3737',
   'mSecToReachLinkFromStart': 21902,
   'linkLength': 31.35},
  {'linkId': 759253715,
   'functionalClass': 4,
   'confidence': 0.96,
   'shape': '54.38744 10.3737 54.38758 10.37369 54.3879 10.37362',
   'mSecToReachLinkFromStart': 30332,
   'linkLength': 51.51},
  {'linkId': 1224249673,
   'functionalClass': 4,
   'confidence': 1.0,
   'shape': '54.3879 10.37362 54.3886 10.37342',
   'mSecToReachLinkFromStart': 43358,
   'linkLength': 79.0},
  {'linkId': 1224249674,
   'functionalClass': 4,
   'confidence': 1.0,
   'shape': '54.3886 10.37342 54.38883 10.37335 54.38902 10.37331',
   'mSe

In [7]:
latlons = [(tp['lon'], tp['lat']) for tp in j['TracePoints']]
# latlons_matched = [(tp['lonMatched'], tp['latMatched']) for tp in j['TracePoints']]

In [8]:
geom = LineString(coordinates=latlons)
feat = Feature(geometry=geom)
feat

{"geometry": {"coordinates": [[10.3747257, 54.3874018], [10.374617, 54.3873829], [10.374693, 54.3873234], [10.374698, 54.3872975], [10.3746879, 54.3873098], [10.3746964, 54.3873083], [10.3746964, 54.3873083], [10.3746528, 54.3872967], [10.3746503, 54.3872982], [10.374655, 54.3872976], [10.3746687, 54.3873047], [10.3744049, 54.3871919], [10.3739579, 54.3870903], [10.3738316, 54.3871143], [10.3737172, 54.387397], [10.3736002, 54.3879267], [10.3735375, 54.3881216], [10.3734749, 54.3882648], [10.3733834, 54.3886038], [10.3732824, 54.3889966], [10.3732605, 54.3892146], [10.3732638, 54.3892388], [10.3732638, 54.3892388], [10.3732657, 54.3892612], [10.3732694, 54.3895146], [10.3728707, 54.3896102], [10.3724973, 54.3895704], [10.3725054, 54.3895635], [10.3725091, 54.3895613], [10.3725139, 54.3895603], [10.3725201, 54.3895588], [10.372524, 54.3895553], [10.3725334, 54.3895485], [10.3725399, 54.3895408], [10.3725556, 54.3895316], [10.3725888, 54.389529], [10.3726065, 54.3895267], [10.3726277, 54

In [9]:
north = [54.34, 10.28]
m = Map(center=north, zoom=11)
m += GeoJSON(data=dict(feat))
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [10]:
m.basemap

{'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
 'max_zoom': 19,
 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}

In [11]:
from utils import build_here_basemap

bm = build_here_basemap(app_id=APP_ID, app_code=APP_CODE)
m2 = Map(center=north, zoom=11, basemap=bm)
m2 += GeoJSON(data=dict(feat))
m2

Map(basemap={'url': 'https://2.traffic.maps.api.here.com/maptile/2.1/traffictile/newest/normal.day/{z}/{x}/{y}…

In [12]:
from ipyleaflet import SplitMapControl

m3 = Map(center=north, zoom=10)
control = SplitMapControl(
    left_layer=basemap_to_tiles(m.basemap),
    right_layer=basemap_to_tiles(m2.basemap))
m3.add_control(control)
m3

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
# latlons = [(tp['lat'], tp['lon']) for tp in j['TracePoints']]
latlons_matched = [(tp['latMatched'], tp['lonMatched']) for tp in j['TracePoints']]

In [14]:
# no add both routes "manually" without GeoJSON
from ipyleaflet import Polyline

polyline = Polyline(locations=latlons, 
                    color="red",
                    fill=False)
m3 += polyline

polyline_matched = Polyline(locations=latlons_matched, 
                            color="blue",
                            fill=False)
m3 += polyline_matched